Name: Behiye Avcı

I hereby declare that I observed the honour code of the university when preparing the homework.

## Solution 16.3-5

#### 16.3

| $S_{t}$ | $S_{t-1}$   |
|------|------|
|  $S_{t}$ = $S_{t-1}$  | p|
| $S_{t}$ = r  | 1-p|

| $y_{t}$ | $S_{t-1}$   |
|------|------|
|  $y_{t}$ = $S_{t-1}$  | q|
| $y_{t}$ = r  | 1-q|

r: random


<img src="img/16_3.png" width="390" height="390">

#### 16.4

| $S_{t}$ | $S_{t-1}$   |$S_{t-1} $  = $a$| $S_{t-1}$= $e$   |
|------|------|------|------|
|  $S_{t}$ = $S_{t-1} + 1$  | p|p|0|
| $S_{t}$ = a  | 1-p|1-p|1-p|
| $S_{t}$ = e  | 0|0|p|


<img src="img/16_4.png">

#### 16.5

States values can be either 0 or 1. If 4 consecutive states have the same value, then the next one should be definitely different. If only 3 consecutive states have the same value, then the next state could be either 1 or 0. In all other situations, the value of the state is the same with former state's value.

<img src="img/16_5.png">

## Solution 18

#### 18.1. Directed Graphical Model

<img src="img/18_1.png" width="370" height="191">

#### 18.2. Undirected Graphical Model

<img src="img/18_2.png" width="370" height="191">

#### Factor Graph 

<img src="img/18_2_2.png" width="370" height="191">

#### 18.3

$p(A, B, D, F, T, L, M, X) = p(F|T, L)p(M)p(T|A)p(B|M)p(X|F)p(L|M)p(D|F, B)p(A)$

If there is N states for each variable, we can determine computing space of each factor as follows:

* $p(F|T, L)$ T and L variables are determines the F. So $N^{2}$ variables required. Also, we need to know N-1 variables for F. Because we will determine the value of the last states by substracting others from 1. In conclusion computing space will be $N^{2}(N-1)$

Following factors will be calculated in the same way:

* $p(M)$ : $N-1$
* $p(T|A)$ : $N(N-1)$
* $p(B|M)$ : $N(N-1)$
* $p(X|F)$ : $N(N-1)$
* $p(L|M)$ : $N(N-1)$
* $p(D|F, B)$ : $N^{2}(N-1)$
* $p(M)$ : $N-1$

    In total the comuting space is $2(N-1)(N+1)^{2}$
 

#### 18.4

#### Answer a

False. There is no responsibility to breake stones even harder

#### Answer b

False. Although we remove X item, we can still reach from A to M in the following way : ATFLM

#### Answer c

False. There is still road from T to L as follows: TFL

#### Answer d

True. The only way from X to L is removed if we do check the probabilty.


## Programming Homework 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [98]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [99]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


### Answer 1

In [100]:
import numpy as np

def func1(N):
    str = "."
    for i in range(0,N):
        idx = currentIndex();
        str = str + alphabet[idx]
    return str;
        
def currentIndex():
    sum = 0;
    x = np.random.random()
    #print(x)
    for i in range(0,len(T[0])):
        sum = sum + float(T[0][i])
        #print(sum)
        if(x < sum):
            return i

In [101]:
K = func1(100)
print(K)

.nsliptlmttspv..nttttr.lnnenrnbprfnwz.ntpyoybnnnitnntrsclntlrinsnirtyrlrnndllmsdngcttnmrowsynslnnlslk


### Answer 2